In [1]:
pip install sqlalchemy sqlite3 pandas


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from sqlalchemy import create_engine
import pandas as pd

# Connect to SQLite database (creates file if it doesn't exist)
engine = create_engine('sqlite:///nutrition.db')


In [ ]:
from sqlalchemy import create_engine, text

#  Step 1: Connect to SQLite database (creates file if it doesn't exist)
engine = create_engine('sqlite:///nutrition.db')

# Step 2: Create tables with exact column names (quoted where needed)
with engine.connect() as conn:
    
    #  Table 1: product_info
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS product_info (
            product_code TEXT PRIMARY KEY,
            product_name TEXT,
            brand TEXT
        );
    """))

    # Table 2: nutrient_info
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS nutrient_info (
            product_code TEXT,
            "energy-kcal_value" FLOAT,
            "energy-kj_value" FLOAT,
            carbohydrates_value FLOAT,
            sugars_value FLOAT,
            fat_value FLOAT,
            "saturated-fat_value" FLOAT,
            proteins_value FLOAT,
            fiber_value FLOAT,
            salt_value FLOAT,
            sodium_value FLOAT,
            "fruits-vegetables-nuts-estimate-from-ingredients_100g" FLOAT,
            "nutrition-score-fr" INTEGER,
            "nova-group" INTEGER,
            FOREIGN KEY(product_code) REFERENCES product_info(product_code)
        );
    """))

    # Table 3: derived_metrics
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS derived_metrics (
            product_code TEXT,
            sugar_to_carb_ratio FLOAT,
            calorie_category TEXT,
            sugar_category TEXT,
            is_ultra_processed TEXT,
            FOREIGN KEY(product_code) REFERENCES product_info(product_code)
        );
    """))


In [ ]:
import pandas as pd

# Load the final cleaned CSV file
df_final = pd.read_csv("data/final_cleaned_data.csv")

# Create df_product for product_info table
df_product = df_final[['product_code', 'product_name', 'brand']].copy()

#Create df_nutrient for nutrient_info table
df_nutrient = df_final[[
    'product_code',
    'energy-kcal_value',
    'energy-kj_value',
    'carbohydrates_value',
    'sugars_value',
    'fat_value',
    'saturated-fat_value',
    'proteins_value',
    'fiber_value',
    'salt_value',
    'sodium_value',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g',
    'nutrition-score-fr',
    'nova-group'
]].copy()

# Create df_derived for derived_metrics table
df_derived = df_final[[
    'product_code',
    'sugar_to_carb_ratio',
    'calorie_category',
    'sugar_category',
    'is_ultra_processed'
]].copy()


In [ ]:
import pandas as pd

#  Step 1: Load the final cleaned CSV
df_final = pd.read_csv("data/final_cleaned_data.csv")

# Step 2: Clean product_code formatting across all rows
# Explanation: product_code is the key used to join tables, so we standardize it
df_final['product_code'] = df_final['product_code'].astype(str).str.strip().str.replace('.0', '', regex=False)


In [ ]:
# Create df_product for product_info table
# Explanation: Select only the columns needed for product_info
df_product = df_final[['product_code', 'product_name', 'brand']].copy()

# Create df_nutrient for nutrient_info table
# Explanation: Select all nutrient-related columns with exact names as per schema
df_nutrient = df_final[[
    'product_code',
    'energy-kcal_value',
    'energy-kj_value',
    'carbohydrates_value',
    'sugars_value',
    'fat_value',
    'saturated-fat_value',
    'proteins_value',
    'fiber_value',
    'salt_value',
    'sodium_value',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g',
    'nutrition-score-fr',
    'nova-group'
]].copy()

# Create df_derived for derived_metrics table
# Explanation: Select derived columns used for categorization and analysis
df_derived = df_final[[
    'product_code',
    'sugar_to_carb_ratio',
    'calorie_category',
    'sugar_category',
    'is_ultra_processed'
]].copy()


In [ ]:
#  Print row counts to verify consistency
print("Rows in df_product:", len(df_product))
print("Rows in df_nutrient:", len(df_nutrient))
print("Rows in df_derived:", len(df_derived))


Rows in df_product: 12000
Rows in df_nutrient: 12000
Rows in df_derived: 12000


In [ ]:
#  Check if all product_codes are consistent across tables
product_codes_product = set(df_product['product_code'])
product_codes_nutrient = set(df_nutrient['product_code'])
product_codes_derived = set(df_derived['product_code'])

#  Find common product_codes
common_codes = product_codes_product & product_codes_nutrient & product_codes_derived

print("Common product_codes across all tables:", len(common_codes))


Common product_codes across all tables: 12000


In [ ]:
from sqlalchemy import create_engine, text

#  Step 1: Connect to SQLite database
engine = create_engine('sqlite:///nutrition.db')

# Step 2: Insert data using the same connection style
with engine.connect() as conn:

    # 🗃️ Insert into product_info table
    for _, row in df_product.iterrows():
        conn.execute(text("""
            INSERT OR IGNORE INTO product_info (product_code, product_name, brand)
            VALUES (:product_code, :product_name, :brand)
        """), {
            'product_code': row['product_code'],
            'product_name': row['product_name'],
            'brand': row['brand']
        })

    #  Insert into nutrient_info table
    for _, row in df_nutrient.iterrows():
        conn.execute(text("""
            INSERT OR IGNORE INTO nutrient_info (
                product_code,
                "energy-kcal_value",
                "energy-kj_value",
                carbohydrates_value,
                sugars_value,
                fat_value,
                "saturated-fat_value",
                proteins_value,
                fiber_value,
                salt_value,
                sodium_value,
                "fruits-vegetables-nuts-estimate-from-ingredients_100g",
                "nutrition-score-fr",
                "nova-group"
            )
            VALUES (
                :product_code,
                :energy_kcal,
                :energy_kj,
                :carbs,
                :sugars,
                :fat,
                :saturated_fat,
                :proteins,
                :fiber,
                :salt,
                :sodium,
                :fvnuts,
                :score_fr,
                :nova
            )
        """), {
            'product_code': row['product_code'],
            'energy_kcal': row['energy-kcal_value'],
            'energy_kj': row['energy-kj_value'],
            'carbs': row['carbohydrates_value'],
            'sugars': row['sugars_value'],
            'fat': row['fat_value'],
            'saturated_fat': row['saturated-fat_value'],
            'proteins': row['proteins_value'],
            'fiber': row['fiber_value'],
            'salt': row['salt_value'],
            'sodium': row['sodium_value'],
            'fvnuts': row['fruits-vegetables-nuts-estimate-from-ingredients_100g'],
            'score_fr': row['nutrition-score-fr'],
            'nova': row['nova-group']
        })

    #  Insert into derived_metrics table
    for _, row in df_derived.iterrows():
        conn.execute(text("""
            INSERT OR IGNORE INTO derived_metrics (
                product_code,
                sugar_to_carb_ratio,
                calorie_category,
                sugar_category,
                is_ultra_processed
            )
            VALUES (
                :product_code,
                :ratio,
                :calorie_cat,
                :sugar_cat,
                :ultra
            )
        """), {
            'product_code': row['product_code'],
            'ratio': row['sugar_to_carb_ratio'],
            'calorie_cat': row['calorie_category'],
            'sugar_cat': row['sugar_category'],
            'ultra': row['is_ultra_processed']
        })


In [ ]:
from sqlalchemy import create_engine, text

# Connect to SQLite database
engine = create_engine('sqlite:///nutrition.db')

#  Insert into product_info
with engine.begin() as conn:
    for _, row in df_product.iterrows():
        conn.execute(text("""
            INSERT OR IGNORE INTO product_info (product_code, product_name, brand)
            VALUES (:product_code, :product_name, :brand)
        """), {
            'product_code': row['product_code'],
            'product_name': row['product_name'],
            'brand': row['brand']
        })

#  Insert into nutrient_info
with engine.begin() as conn:
    for _, row in df_nutrient.iterrows():
        conn.execute(text("""
            INSERT OR IGNORE INTO nutrient_info (
                product_code,
                "energy-kcal_value",
                "energy-kj_value",
                carbohydrates_value,
                sugars_value,
                fat_value,
                "saturated-fat_value",
                proteins_value,
                fiber_value,
                salt_value,
                sodium_value,
                "fruits-vegetables-nuts-estimate-from-ingredients_100g",
                "nutrition-score-fr",
                "nova-group"
            )
            VALUES (
                :product_code,
                :energy_kcal,
                :energy_kj,
                :carbs,
                :sugars,
                :fat,
                :saturated_fat,
                :proteins,
                :fiber,
                :salt,
                :sodium,
                :fvnuts,
                :score_fr,
                :nova
            )
        """), {
            'product_code': row['product_code'],
            'energy_kcal': row['energy-kcal_value'],
            'energy_kj': row['energy-kj_value'],
            'carbs': row['carbohydrates_value'],
            'sugars': row['sugars_value'],
            'fat': row['fat_value'],
            'saturated_fat': row['saturated-fat_value'],
            'proteins': row['proteins_value'],
            'fiber': row['fiber_value'],
            'salt': row['salt_value'],
            'sodium': row['sodium_value'],
            'fvnuts': row['fruits-vegetables-nuts-estimate-from-ingredients_100g'],
            'score_fr': row['nutrition-score-fr'],
            'nova': row['nova-group']
        })

# Insert into derived_metrics
with engine.begin() as conn:
    for _, row in df_derived.iterrows():
        conn.execute(text("""
            INSERT OR IGNORE INTO derived_metrics (
                product_code,
                sugar_to_carb_ratio,
                calorie_category,
                sugar_category,
                is_ultra_processed
            )
            VALUES (
                :product_code,
                :ratio,
                :calorie_cat,
                :sugar_cat,
                :ultra
            )
        """), {
            'product_code': row['product_code'],
            'ratio': row['sugar_to_carb_ratio'],
            'calorie_cat': row['calorie_category'],
            'sugar_cat': row['sugar_category'],
            'ultra': row['is_ultra_processed']
        })


In [ ]:
import pandas as pd
import sqlite3

#  Connect to the SQLite database
conn = sqlite3.connect("nutrition.db")

# Export product_info table
df_product_info = pd.read_sql_query("SELECT * FROM product_info", conn)
df_product_info.to_csv("D:/Projects/Chococrunch_Analytics/env/product_info.csv", index=False)

#  Export nutrient_info table
df_nutrient_info = pd.read_sql_query("SELECT * FROM nutrient_info", conn)
df_nutrient_info.to_csv("D:/Projects/Chococrunch_Analytics/env/nutrient_info.csv", index=False)

#  Export derived_metrics table
df_derived_metrics = pd.read_sql_query("SELECT * FROM derived_metrics", conn)
df_derived_metrics.to_csv("D:/Projects/Chococrunch_Analytics/env/derived_metrics.csv", index=False)

# Close the connection
conn.close()


In [ ]:
############27 SQL QUERIES 

In [12]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("nutrition.db")

# Helper function to run and display SQL queries
def run_query(query):
    return pd.read_sql_query(query, conn)


In [ ]:
###product_info Queries
# 1️ Count products per brand
# Explanation: Shows how many products each brand has listed
run_query("""
    SELECT brand, COUNT(*) AS product_count
    FROM product_info
    GROUP BY brand
""")

,brand,product_count
0,None,8
1,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",1
2,"1 x Lidl 04.24, Alesto",1
3,"1001 Delights (Lidl), Envia, Lidl, Milbona, Mi...",1
4,"1540 Vestby, Norwegen, Oluf Lorentzen AS, Tove...",1
...,...,...
4016,كنور,1
4017,كوسومار,1
4018,لاسيگون,1
4019,نلزه,1


In [14]:
# 2️ Count unique products per brand
# Explanation: Ensures product_code uniqueness per brand
run_query("""
    SELECT brand, COUNT(DISTINCT product_code) AS unique_products
    FROM product_info
    GROUP BY brand
""")

,brand,unique_products
0,None,8
1,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",1
2,"1 x Lidl 04.24, Alesto",1
3,"1001 Delights (Lidl), Envia, Lidl, Milbona, Mi...",1
4,"1540 Vestby, Norwegen, Oluf Lorentzen AS, Tove...",1
...,...,...
4016,كنور,1
4017,كوسومار,1
4018,لاسيگون,1
4019,نلزه,1


In [15]:
# 3️ Top 5 brands by product count
# Explanation: Identifies the most prolific brands
run_query("""
    SELECT brand, COUNT(*) AS product_count
    FROM product_info
    GROUP BY brand
    ORDER BY product_count DESC
    LIMIT 5
""")

,brand,product_count
0,Brand_Unavailable,652
1,Hacendado,352
2,Tesco,269
3,Carrefour,146
4,Bjorg,98


In [16]:
# 4️ Products with missing product name
# Explanation: Checks for incomplete product entries
run_query("""
    SELECT *
    FROM product_info
    WHERE product_name IS NULL OR product_name = ''
""")

,product_code,product_name,brand
0,6111242100985,None,Jaouda
1,6111099003897,None,lilia
2,6111128000460,None,aïn Saiss
3,20005733,None,"Alesto,Lidl"
4,8425197712024,None,Maruja
...,...,...,...
298,5900617002266,None,Sante
299,8410376052633,None,Gullón
300,8008698002025,None,Schär
301,168069,None,Marks & Spencer


In [17]:
# 5️ Number of unique brands
# Explanation: Total distinct brands in the dataset
run_query("""
    SELECT COUNT(DISTINCT brand) AS unique_brand_count
    FROM product_info
""")

,unique_brand_count
0,4020


In [18]:
# 6️ Products with code starting with '3'
# Explanation: Filters products based on code prefix
run_query("""
    SELECT *
    FROM product_info
    WHERE product_code LIKE '3%'
""")

,product_code,product_name,brand
0,3274080005003,eau de source,Cristaline
1,3046920029759,edelbitterschokolade mild 90,Lindt&Sprüngli
2,3046920028004,excellence 70 cocoa intense dark,Lindt
3,3017620425035,nutella,Ferrero
4,3175680011480,sésame,Gerblé
...,...,...,...
3656,3362600003100,cigares fourrés cacao,Brand_Unavailable
3657,3185200001327,croutons de pains croustillants,Tipiak
3658,3450601046254,3450601046254,l'arbre vert
3659,3088541015209,unknown,Lune de Miel


In [19]:
###nutrient_info Queries
# 7️ Top 10 products with highest energy-kcal_value
# Explanation: Identifies most calorie-dense products
run_query("""
    SELECT product_code, "energy-kcal_value"
    FROM nutrient_info
    ORDER BY "energy-kcal_value" DESC
    LIMIT 10
""")

,product_code,energy-kcal_value
0,8480000054647,45793.499044
1,5010029221701,14770.554493
2,6111259090132,5280.000000
3,3088543506255,1600.000000
4,72486002205,990.000000
5,3175681297098,974.358974
6,36000291452,904.000000
7,6111099000599,900.000000
8,6111024001516,900.000000
9,6111024002186,900.000000


In [20]:
# 8️ Average sugars_value per nova-group
# Explanation: Measures sugar content across processing levels
run_query("""
    SELECT "nova-group", AVG(sugars_value) AS avg_sugar
    FROM nutrient_info
    GROUP BY "nova-group"
""")

,nova-group,avg_sugar
0,1,5.969890
1,2,16.706155
2,3,6.893368
3,4,12.700990


In [21]:
# 9️ Count products with fat_value > 20g
# Explanation: Flags high-fat products
run_query("""
    SELECT COUNT(*) AS high_fat_products
    FROM nutrient_info
    WHERE fat_value > 20
""")

,high_fat_products
0,2923


In [22]:
#  Average carbohydrates_value per product
# Explanation: Overall carb content across products
run_query("""
    SELECT AVG(carbohydrates_value) AS avg_carbs
    FROM nutrient_info
""")

,avg_carbs
0,29.648376


In [23]:
# 1️1️ Products with sodium_value > 1g
# Explanation: Identifies high-sodium items
run_query("""
    SELECT product_code, sodium_value
    FROM nutrient_info
    WHERE sodium_value > 1
""")

,product_code,sodium_value
0,6111035002175,26.000
1,6111035000058,26.000
2,3274080005003,21.000
3,50184453,4.320
4,6111035001635,48.000
...,...,...
666,72486002205,2.070
667,5000328149035,1.144
668,6194001800302,36.700
669,863699000108,85.000


In [24]:

# 1️2️ Count products with non-zero fruits-vegetables-nuts content
# Explanation: Measures nutritional richness
run_query("""
    SELECT COUNT(*) AS non_zero_fvn
    FROM nutrient_info
    WHERE "fruits-vegetables-nuts-estimate-from-ingredients_100g" > 0
""")

,non_zero_fvn
0,6526


In [25]:
# 1️3️ Products with energy-kcal_value > 500
# Explanation: Flags very high-calorie products
run_query("""
    SELECT product_code, "energy-kcal_value"
    FROM nutrient_info
    WHERE "energy-kcal_value" > 500
""")

,product_code,energy-kcal_value
0,6111099003897,675.0
1,3046920029759,592.0
2,3046920028004,566.0
3,3017620425035,539.0
4,3046920028363,584.0
...,...,...
1735,6111203005038,514.0
1736,5000119007247,538.0
1737,4066447423761,824.0
1738,7622201768836,558.0


In [26]:
###derived_metrics Queries
# 1️4️ Count products per calorie_category
# Explanation: Distribution of products by calorie level
run_query("""
    SELECT calorie_category, COUNT(*) AS product_count
    FROM derived_metrics
    GROUP BY calorie_category
""")

,calorie_category,product_count
0,High,4826
1,Low,3048
2,Moderate,4126


In [27]:

# 1️5️ Count of High Sugar products
# Explanation: Total products labeled as high sugar
run_query("""
    SELECT COUNT(*) AS high_sugar_count
    FROM derived_metrics
    WHERE sugar_category = 'High Sugar'
""")

,high_sugar_count
0,2514


In [28]:
# 1️6 Average sugar_to_carb_ratio for High Calorie products
# Explanation: Measures sugar density in high-calorie items
run_query("""
    SELECT AVG(sugar_to_carb_ratio) AS avg_ratio
    FROM derived_metrics
    WHERE calorie_category = 'High Calorie'
""")

,avg_ratio
0,None


In [29]:

# 1️7️ Products that are both High Calorie and High Sugar
# Explanation: Flags nutritionally risky products
run_query("""
    SELECT *
    FROM derived_metrics
    WHERE calorie_category = 'High Calorie'
      AND sugar_category = 'High Sugar'
""")

,product_code,sugar_to_carb_ratio,calorie_category,sugar_category,is_ultra_processed


In [30]:
# 1️8️ Number of products marked as ultra-processed
# Explanation: Total products flagged as ultra-processed
run_query("""
    SELECT COUNT(*) AS ultra_processed_count
    FROM derived_metrics
    WHERE is_ultra_processed = 'Yes'
""")

,ultra_processed_count
0,7574


In [31]:
# 1️9️ Products with sugar_to_carb_ratio > 0.7
# Explanation: Identifies sugar-dense products
run_query("""
    SELECT product_code, sugar_to_carb_ratio
    FROM derived_metrics
    WHERE sugar_to_carb_ratio > 0.7
""")

,product_code,sugar_to_carb_ratio
0,5449000054227,1.000000
1,3046920028004,0.882353
2,3017620425035,0.979130
3,5449000147417,1.827957
4,6111162000839,1.027230
...,...,...
4158,6111242923850,0.990909
4159,7622201768836,0.910714
4160,4056489774747,0.939498
4161,5010478014510,0.750000


In [32]:
# 2️0️ Average sugar_to_carb_ratio per calorie_category
# Explanation: Compares sugar density across calorie levels
run_query("""
    SELECT calorie_category, AVG(sugar_to_carb_ratio) AS avg_ratio
    FROM derived_metrics
    GROUP BY calorie_category
""")

,calorie_category,avg_ratio
0,High,0.426422
1,Low,17.501839
2,Moderate,0.430583


In [33]:
####Join Queries
# 2️1️ Top 5 brands with most High Calorie products
# Explanation: Combines product_info and derived_metrics
run_query("""
    SELECT pi.brand, COUNT(*) AS high_calorie_count
    FROM product_info pi
    JOIN derived_metrics dm ON pi.product_code = dm.product_code
    WHERE dm.calorie_category = 'High Calorie'
    GROUP BY pi.brand
    ORDER BY high_calorie_count DESC
    LIMIT 5
""")

,brand,high_calorie_count


In [34]:
# Check distinct calorie_category values
run_query("""
    SELECT DISTINCT calorie_category
    FROM derived_metrics
""")


,calorie_category
0,Low
1,Moderate
2,High


In [35]:
run_query("""
    SELECT pi.brand, COUNT(*) AS high_calorie_count
    FROM product_info pi
    JOIN derived_metrics dm ON TRIM(pi.product_code) = TRIM(dm.product_code)
    WHERE dm.calorie_category = 'High'
    GROUP BY pi.brand
    ORDER BY high_calorie_count DESC
    LIMIT 5
""")


,brand,high_calorie_count
0,Hacendado,150
1,Brand_Unavailable,134
2,Tesco,86
3,Carrefour,76
4,Kellogg's,60


In [36]:
# 2️2️ Average energy-kcal_value for each calorie_category
# Explanation: Combines nutrient_info and derived_metrics
run_query("""
    SELECT dm.calorie_category, AVG(ni."energy-kcal_value") AS avg_kcal
    FROM nutrient_info ni
    JOIN derived_metrics dm ON ni.product_code = dm.product_code
    GROUP BY dm.calorie_category
""")

,calorie_category,avg_kcal
0,High,492.878067
1,Low,48.341022
2,Moderate,198.171284


In [37]:
# 2️3️ Count of ultra-processed products per brand
# Explanation: Combines product_info and derived_metrics
run_query("""
    SELECT pi.brand, COUNT(*) AS ultra_processed_count
    FROM product_info pi
    JOIN derived_metrics dm ON pi.product_code = dm.product_code
    WHERE dm.is_ultra_processed = 'Yes'
    GROUP BY pi.brand
""")

,brand,ultra_processed_count
0,None,8
1,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",1
2,"1 x Lidl 04.24, Alesto",1
3,"1001 Delights (Lidl), Envia, Lidl, Milbona, Mi...",1
4,2 KEEP NATURAL BITES,1
...,...,...
2707,سنطرال,1
2708,كنور,1
2709,كوسومار,1
2710,لاسيگون,1


In [38]:
# 2️4️ Products with High Sugar and High Calorie along with brand
# Explanation: Multi-table join for risky products
run_query("""
    SELECT pi.product_name, pi.brand, dm.calorie_category, dm.sugar_category
    FROM product_info pi
    JOIN derived_metrics dm ON pi.product_code = dm.product_code
    WHERE dm.calorie_category = 'High Calorie'
      AND dm.sugar_category = 'High Sugar'
""")

,product_name,brand,calorie_category,sugar_category


In [39]:
run_query("""
    SELECT DISTINCT sugar_category
    FROM derived_metrics
""")


,sugar_category
0,Moderate Sugar
1,Low Sugar
2,High Sugar


In [40]:
run_query("""
    SELECT pi.product_name, pi.brand, dm.calorie_category, dm.sugar_category
    FROM product_info pi
    JOIN derived_metrics dm ON TRIM(pi.product_code) = TRIM(dm.product_code)
    WHERE dm.calorie_category = 'High'
      AND dm.sugar_category = 'High Sugar'
""")


,product_name,brand,calorie_category,sugar_category
0,prince goût chocolat,Lu,High,High Sugar
1,excellence 70 cocoa intense dark,Lindt,High,High Sugar
2,nutella,Ferrero,High,High Sugar
3,sésame,Gerblé,High,High Sugar
4,nutella,Ferrero,High,High Sugar
...,...,...,...,...
1971,milka chips ahoy chocolate,None,High,High Sugar
1972,None,"Mac Kennedy, MCEnnedy",High,High Sugar
1973,nairns dark chocolate chip oat biscuits,Nairn’s,High,High Sugar
1974,drumstick squashies,Swizzels,High,High Sugar


In [41]:
# 2️5️ Average sugar content per brand for ultra-processed products
# Explanation: Combines nutrient_info, derived_metrics, and product_info
run_query("""
    SELECT pi.brand, AVG(ni.sugars_value) AS avg_sugar
    FROM product_info pi
    JOIN nutrient_info ni ON pi.product_code = ni.product_code
    JOIN derived_metrics dm ON pi.product_code = dm.product_code
    WHERE dm.is_ultra_processed = 'Yes'
    GROUP BY pi.brand
""")

,brand,avg_sugar
0,None,20.325
1,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",5.000
2,"1 x Lidl 04.24, Alesto",7.700
3,"1001 Delights (Lidl), Envia, Lidl, Milbona, Mi...",12.800
4,2 KEEP NATURAL BITES,26.000
...,...,...
2707,سنطرال,4.100
2708,كنور,0.500
2709,كوسومار,100.000
2710,لاسيگون,7.300


In [42]:
# 2️6️ Number of products with fruits/vegetables/nuts content in each calorie_category
# Explanation: Combines nutrient_info and derived_metrics
run_query("""
    SELECT dm.calorie_category, COUNT(*) AS fvn_count
    FROM nutrient_info ni
    JOIN derived_metrics dm ON ni.product_code = dm.product_code
    WHERE ni."fruits-vegetables-nuts-estimate-from-ingredients_100g" > 0
    GROUP BY dm.calorie_category
""")

,calorie_category,fvn_count
0,High,2411
1,Low,1732
2,Moderate,2383


In [ ]:
# 2️7️ Top 5 products by sugar_to_carb_ratio with their calorie and sugar category
# Explanation: Highlights extreme sugar density
run_query("""
    SELECT dm.product_code, dm.sugar_to_carb_ratio, dm.calorie_category, dm.sugar_category
    FROM derived_metrics dm
    ORDER BY dm.sugar_to_carb_ratio DESC
    LIMIT 5
""")

,product_code,sugar_to_carb_ratio,calorie_category,sugar_category
0,5000442007617,51250.000000,Low,Low Sugar
1,41500007229,87.234043,Low,Low Sugar
2,6111184003283,29.285714,Low,Low Sugar
3,8402001021428,20.500000,Low,Low Sugar
4,8480000511263,19.523810,Low,Low Sugar


In [45]:
#####################For Powerbi
# Load product_info from SQLite
import pandas as pd
import sqlite3

conn = sqlite3.connect("nutrition.db")
df_product_info = pd.read_sql_query("SELECT * FROM product_info", conn)


In [46]:

#  Check for missing values
df_product_info.isnull().sum()



product_code      0
product_name    303
brand             8
dtype: int64

In [47]:
# Check for duplicate product_code (should be unique)
df_product_info['product_code'].duplicated().sum()



np.int64(0)

In [50]:
#  Check total row count
len(df_product_info)

12000

In [49]:
df_product_info['product_name'] = df_product_info['product_name'].fillna('Unknown Product')
df_product_info['brand'] = df_product_info['brand'].fillna('Unknown Brand')


In [51]:
print("product_info types:\n", df_product_info.dtypes)
print("nutrient_info types:\n", df_nutrient_info.dtypes)
print("derived_metrics types:\n", df_derived_metrics.dtypes)


product_info types:
 product_code    object
product_name    object
brand           object
dtype: object
nutrient_info types:
 product_code                                              object
energy-kcal_value                                        float64
energy-kj_value                                          float64
carbohydrates_value                                      float64
sugars_value                                             float64
fat_value                                                float64
saturated-fat_value                                      float64
proteins_value                                           float64
fiber_value                                              float64
salt_value                                               float64
sodium_value                                             float64
fruits-vegetables-nuts-estimate-from-ingredients_100g    float64
nutrition-score-fr                                         int64
nova-group                   

In [ ]:
df_product_info.to_csv("data/product_info.csv", index=False)
df_nutrient_info.to_csv("data/nutrient_info.csv", index=False)
df_derived_metrics.to_csv("data/derived_metrics.csv", index=False)


In [ ]:
# Load the final cleaned CSV file
df_product_info = pd.read_csv("data/product_info.csv")

In [62]:
df_product_info_clean.loc[[4098, 6212]]



,product_code,product_name,brand
4098,3182180060290,tartines de pain multicrales,Pasquier
6212,8714100880142,knorr bouillon de poule bio 60g,Knorr


In [ ]:
df_product_info['product_name'] = df_product_info['product_name'].apply(
    lambda x: x.encode('ascii', 'ignore').decode() if pd.notnull(x) else x
)


In [ ]:
df_product_info.shape

(12000, 3)

In [ ]:
df_product_info['product_name'] = df_product_info['product_name'].str.strip()
df_product_info['brand'] = df_product_info['brand'].str.strip()


In [ ]:
df_product_info.loc[[4098, 6212]]


,product_code,product_name,brand
4098,3182180060290,tartines de pain multicrales,Pasquier
6212,8714100880142,knorr bouillon de poule bio 60g,Knorr


In [ ]:
df_product_info.to_csv("D:/Projects/Chococrunch_Analytics/env/product_info_clean.csv", index=False)

In [71]:
df_product_info.shape

(12000, 3)

In [70]:
df_product_info['product_name'] = df_product_info['product_name'].fillna('Unknown Product')


In [ ]:
df_product_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_code  12000 non-null  int64 
 1   product_name  11972 non-null  object
 2   brand         12000 non-null  object
dtypes: int64(1), object(2)
memory usage: 281.4+ KB


In [ ]:
import pandas as pd
df_product_info = pd.read_csv("data/product_info.csv")


In [ ]:
print("Missing:", df_product_info['product_code'].isnull().sum())
print("Duplicates:", df_product_info['product_code'].duplicated().sum())
print("Type:", df_product_info['product_code'].dtype)


Missing: 0
Duplicates: 0
Type: int64


In [ ]:
df_product_info['product_code'] = df_product_info['product_code'].astype(str).str.strip()


In [ ]:
print("Missing:", df_product_info['product_name'].isnull().sum())
print("Type:", df_product_info['product_name'].dtype)


Missing: 28
Type: object


In [ ]:
df_product_info['product_name'] = df_product_info['product_name'].fillna('Unknown Product')


In [ ]:
print("Missing:", df_product_info['product_name'].isnull().sum())
print("Type:", df_product_info['product_name'].dtype)

Missing: 0
Type: object


In [82]:
import sqlite3

conn = sqlite3.connect("nutrition.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print(tables)


[('product_info',), ('nutrient_info',), ('derived_metrics',)]
